In [56]:
!uv pip install tensorflow tensorflow-datasets tensorflow-hub

Audited 3 packages in 3ms


In [57]:
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

import tensorflow as tf
import tensorflow_hub as hub 
import json
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout

# GPU setup
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    try:
        for device in physical_devices:
            tf.config.experimental.set_memory_growth(device, True)
        print(f"✅ GPU détecté: {len(physical_devices)} GPU(s)")
    except RuntimeError as e:
        print(f"⚠️ Erreur GPU: {e}")
else:
    print("⚠️ CPU uniquement")

⚠️ CPU uniquement


In [58]:
import pandas as pd

# 2. CHARGEMENT ET AUGMENTATION DU DATASET (Depuis dataset.csv)
def load_and_augment_data(csv_path='dataset.csv', target_samples_per_class=50):
    print(f"📂 Chargement de {csv_path}...")
    try:
        df = pd.read_csv(csv_path)
        # Nettoyage des espaces éventuels
        df['type'] = df['type'].astype(str).str.strip()
        df['sentence'] = df['sentence'].astype(str).str.strip()
    except FileNotFoundError:
        print(f"❌ Fichier {csv_path} introuvable !")
        return np.array([], dtype=str), np.array([], dtype=int)

    # Vérification des classes
    unique_types = df['type'].unique()
    print(f"Classes trouvées dans le CSV : {unique_types}")
    
    # Récupération des classes globales
    if 'classes' not in globals() or not classes:
        print("⚠️ Variable 'classes' vide ou non définie. Utilisation des classes du CSV.")
        current_classes = unique_types
    else:
        current_classes = classes
        
    print(f"Classes utilisées pour l'augmentation : {current_classes}")
    
    data_inputs = []
    data_labels = []

    for cls in current_classes:
        # Filtrer les exemples de cette classe
        class_samples = df[df['type'] == cls]['sentence'].tolist()
        
        if not class_samples:
            print(f"⚠️ Aucune donnée pour la classe '{cls}' dans le CSV.")
            continue
            
        # On boucle jusqu'à atteindre le nombre cible
        count = 0
        while count < target_samples_per_class:
            for sample in class_samples:
                if count >= target_samples_per_class:
                    break
                
                # Variation 1: Original
                text = sample
                
                # Variation 2: Minuscule
                if count % 3 == 1:
                    text = sample.lower()
                
                # Variation 3: Ajout de politesse (bruit)
                if count % 3 == 2:
                    politesse = np.random.choice(["Bonjour, ", "Svp, ", "J'aimerais savoir ", ""])
                    text = f"{politesse}{sample.lower()}"

                data_inputs.append(text)
                # Use class_to_id if available, else create on fly or skip
                if 'class_to_id' in globals() and cls in class_to_id:
                    data_labels.append(class_to_id[cls])
                else:
                    # Fallback if class_to_id missing (should not happen if cells run in order)
                    data_labels.append(0) 
                count += 1
                
    # Mélange
    if not data_inputs:
        print("❌ Erreur: Aucun dataset généré.")
        return np.array([], dtype=str), np.array([], dtype=int)

    indices = np.arange(len(data_inputs))
    np.random.shuffle(indices)
    data_inputs = np.array(data_inputs)[indices]
    data_labels = np.array(data_labels)[indices]
    
    print(f"✅ Dataset augmenté : {len(data_inputs)} exemples au total.")
    return data_inputs, data_labels

# Exécution
inputs, labels = load_and_augment_data('dataset.csv', target_samples_per_class=60)

if len(inputs) == 0:
    raise ValueError("Le dataset est vide. Vérifiez que les cellules précédentes ont été exécutées et que le CSV est correct.")

# Split train/val
split_idx = int(0.8 * len(inputs))
train_inputs, val_inputs = inputs[:split_idx], inputs[split_idx:]
train_labels, val_labels = labels[:split_idx], labels[split_idx:]

# Datasets TF
BATCH_SIZE = 16
def clean_text(text):
    text = tf.strings.lower(text)
    return text

# Force types to avoid float64 inference on empty (though we raise error now)
train_ds = tf.data.Dataset.from_tensor_slices((train_inputs, train_labels)).batch(BATCH_SIZE).map(lambda x, y: (clean_text(x), y))
val_ds = tf.data.Dataset.from_tensor_slices((val_inputs, val_labels)).batch(BATCH_SIZE).map(lambda x, y: (clean_text(x), y))

📂 Chargement de dataset.csv...
Classes trouvées dans le CSV : ['carte_identite' 'passeport' 'permis_conduire' 'carte_grise' 'etat_civil'
 'amende' 'impots' 'sante' 'allocations' 'rendez_vous_prefecture']
Classes utilisées pour l'augmentation : ['carte_identite', 'passeport', 'acte_naissance', 'certificat_residence', 'immatriculation', 'permis_conduire', 'casier_judiciaire', 'autre']
⚠️ Aucune donnée pour la classe 'acte_naissance' dans le CSV.
⚠️ Aucune donnée pour la classe 'certificat_residence' dans le CSV.
⚠️ Aucune donnée pour la classe 'immatriculation' dans le CSV.
⚠️ Aucune donnée pour la classe 'casier_judiciaire' dans le CSV.
⚠️ Aucune donnée pour la classe 'autre' dans le CSV.
✅ Dataset augmenté : 180 exemples au total.


In [59]:
# 4. MODÈLE DE CLASSIFICATION (Native TensorFlow)
# Utilisation de TextVectorization + Embedding pour éviter les dépendances 
# problématiques (tensorflow-text) sur ROCm et supporter le français.

VOCAB_SIZE = 1000
SEQUENCE_LENGTH = 25

# Création de la couche de vectorisation
vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH,
    name='vectorizer'
)

# Adaptation au vocabulaire du dataset d'entraînement
print("🔄 Adaptation du vocabulaire...")
# On extrait uniquement le texte du dataset pour l'adaptation
train_text_only = train_ds.map(lambda x, y: x)
vectorizer.adapt(train_text_only)
print(f"✅ Vocabulaire adapté (Taille: {len(vectorizer.get_vocabulary())})")

# Architecture du modèle
input_layer = Input(shape=(), dtype=tf.string, name='input_text')

# Vectorisation
x = vectorizer(input_layer)

# Embedding (Apprentissage d'une représentation vectorielle pour chaque mot)
x = tf.keras.layers.Embedding(input_dim=VOCAB_SIZE, output_dim=64, name='embedding')(x)

# Agrégation des vecteurs de mots (GlobalAveragePooling1D est simple et efficace)
x = tf.keras.layers.GlobalAveragePooling1D(name='pooling')(x)

# Classification head
x = Dense(64, activation='relu', name='dense_1')(x)
x = Dropout(0.3)(x)
x = Dense(32, activation='relu', name='dense_2')(x)
outputs = Dense(10, activation='softmax', name='classification')(x)

model = Model(inputs=input_layer, outputs=outputs)
print("🏗️ Architecture définie")

🔄 Adaptation du vocabulaire...
✅ Vocabulaire adapté (Taille: 42)
🏗️ Architecture définie
🏗️ Architecture définie


2025-12-05 01:43:14.108751: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [60]:
# 5. COMPILATION ET ENTRAÎNEMENT
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# Callbacks
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy', patience=3, restore_best_weights=True, verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', factor=0.5, patience=2, min_lr=1e-7, verbose=1
    )
]

# Entraînement
print("🚀 Début de l'entraînement...")
history = model.fit(
    train_ds,
    epochs=15,
    validation_data=val_ds,
    callbacks=callbacks,
    verbose=1
)

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_text (InputLayer)         │ (None)                 │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vectorizer (TextVectorization)  │ (None, 25)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 25, 64)         │        64,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pooling                         │ (None, 64)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ classification (Dense)          │ (None, 10)             │           330 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 70,570 (275.66 KB)

 Trainable params: 70,570 (275.66 KB)

 Non-trainable params: 0 (0.00 B)

🚀 Début de l'entraînement...
Epoch 1/15


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.0069 - loss: 2.3032 - val_accuracy: 0.0000e+00 - val_loss: 2.3015 - learning_rate: 2.0000e-05
Epoch 2/15
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.0069 - loss: 2.3032 - val_accuracy: 0.0000e+00 - val_loss: 2.3015 - learning_rate: 2.0000e-05
Epoch 2/15
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0139 - loss: 2.2997 - val_accuracy: 0.0000e+00 - val_loss: 2.2992 - learning_rate: 2.0000e-05
Epoch 3/15
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0139 - loss: 2.2997 - val_accuracy: 0.0000e+00 - val_loss: 2.2992 - learning_rate: 2.0000e-05
Epoch 3/15
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0069 - loss: 2.2991 - val_accuracy: 0.0000e+00 - val_loss: 2.2968 - learning_rate: 2.0000e-05
Epoch 4/15
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0069 - loss: 2.2991 - val_accuracy: 0.0000e+00 - val_loss: 2.2968 - learning_rate: 2.0000e-05
Epoch 4/15
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0486 - loss

In [61]:
# 8. SAUVEGARDE DU MODÈLE
model.save('models/admin_request_classifier', save_format='tf')
print("💾 Modèle sauvegardé: models/admin_request_classifier")

# 9. EXPORT POUR INFÉRENCE (TensorFlow.js compatible)
print("\n🔄 Conversion pour TensorFlow.js...")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open('models/admin_classifier.tflite', 'wb') as f:
    f.write(tflite_model)
print("✅ Modèle TFLite sauvegardé: models/admin_classifier.tflite")

print("\n🎉 PRÊT POUR DÉPLOIEMENT !")
print("Fichiers générés:")
print("  - models/admin_request_classifier/ (TensorFlow SavedModel)")
print("  - models/admin_classifier.tflite (TensorFlow Lite)")
print("  - data/admin_requests_dataset.json")

ValueError: The `save_format` argument is deprecated in Keras 3. Please remove this argument and pass a file path with either `.keras` or `.h5` extension.Received: save_format=tf